In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
os.chdir("/home/reffert/DeepAR_InfluenzaForecast")
from PythonFiles.model import model,preprocessing
from PythonFiles.Configuration import Configuration
config = Configuration()
population_df = pd.read_csv("population.csv", sep=",")
influenza_df = pd.read_csv("Influenza.csv", sep=",")
geo_df = pd.read_csv("AdjacentMatrix.csv", sep=",")

In [60]:
population_locations = population_df.Stichtag.unique()
influenza_locations = influenza_df.location.unique()
renaming_dict = dict()
population_df["Location"] = population_df["Stichtag"].copy()
population_df.loc[population_df.Location.str.contains(" kreisfreie Stadt"), "Location"] = population_df.loc[population_df.Location.str.contains(" kreisfreie Stadt"),
                                                                                                           "Location"].apply(lambda s: 'SK '+s)
population_df['Location'] = population_df.Location.str.replace(" kreisfreie Stadt" , "")
for lk_indicator in ["-Kreis", " Kreis", " Landkreis", "kreis"]:
    population_df.loc[population_df.Location.str.contains(lk_indicator), "Location"] = population_df.loc[population_df.Location.str.contains(lk_indicator),
                                                                                                           "Location"].apply(lambda s: 'LK '+s)
    population_df['Location'] = population_df.Location.str.replace(lk_indicator , "")
#print(population_df.loc[~(population_df.Location.str.contains("LK") | population_df.Location.str.contains("SK"))])
#print(population_df)
unwanted_strings = [f"{i}" for i in range(0,10)]+[".", "  (bis )", " (bis )"]
for unwanted_string in unwanted_strings:
    population_df.Location = population_df.Location.apply(lambda s: s.replace(unwanted_string, "") if unwanted_string in s else s)
    

renaming_dict = {}
for location in population_df.Location:
    if location in influenza_locations:
        renaming_dict[location] = location
    else:
        renaming_dict[location] = "None"
        
not_matching_dict = {k:v for k, v in zip(renaming_dict.keys(), renaming_dict.values()) if (v=="None")}
print(not_matching_dict)

{'LK Osterode am Harz': 'None', 'SK Hannover': 'None', 'LK Region Hannover': 'None', 'LK Hannover': 'None', 'LK Heide': 'None', 'SK Oldenburg (Oldenburg)': 'None', 'SK Mülheim an der Ruhr': 'None', 'LK Rhein Neuss': 'None', 'SK Aachen': 'None', 'LK Städteregion Aachen': 'None', 'LK Aachen': 'None', 'LK Rhein-Erft': 'None', 'LK Oberbergischer': 'None', 'LK Rheinisch-Bergischer': 'None', 'LK Rhein-Sieg': 'None', 'LK Ennepe-Ruhr': 'None', 'LK Hochsauerland': 'None', 'LK Märkischer': 'None', 'SK Offenbach am Main': 'None', 'LK Hochtaunus': 'None', 'LK Main-Kinzig': 'None', 'LK Main-Taunus': 'None', 'LK Odenwald': 'None', 'LK Rheingau-Taunus': 'None', 'LK Wetterau': 'None', 'LK Lahn-Dill': 'None', 'LK Vogelsberg': 'None', 'LK Schwalm-Eder': 'None', 'LK Werra-Meißner': 'None', 'LK Altenkirchen (Westerwald)': 'None', 'LK Rhein-Hunsrück': 'None', 'LK Rhein-Lahn': 'None', 'LK Westerwald': 'None', 'LK Eifel Bitburg-Prüm': 'None', 'SK Frankenthal (Pfalz)': 'None', 'SK Landau in der Pfalz': 'None'

In [3]:
# save the entries of the renaming dict where we have no match or at least 2 matches
not_matching_dict = {k:v for k, v in zip(renaming_dict.keys(), renaming_dict.values()) if ((len(v)==0) | (len(v)>1))}
# also save the entries where we have exactly one match (as it's supposed to be)
correctly_matched_dict = {k:v for k, v in zip(renaming_dict.keys(), renaming_dict.values()) if len(v) == 1}

print(sorted(not_matching_dict))
print(not_matching_dict)

['Aachen Landkreis', 'Aachen kreisfreie Stadt', 'Ahrweiler Landkreis', 'Aichach-Friedberg Landkreis', 'Altenburger Land Landkreis', 'Altenkirchen (Westerwald) Landkreis', 'Altötting Landkreis', 'Alzey-Worms Landkreis', 'Amberg kreisfreie Stadt', 'Amberg-Sulzbach Landkreis', 'Ammerland Landkreis', 'Anhalt-Bitterfeld Landkreis', 'Anhalt-Zerbst Landkreis', 'Annaberg Landkreis', 'Ansbach Landkreis', 'Ansbach kreisfreie Stadt', 'Aschaffenburg Landkreis', 'Aschaffenburg kreisfreie Stadt', 'Aschersleben-Staßfurt Landkreis', 'Aue-Schwarzenberg Landkreis', 'Augsburg Landkreis', 'Augsburg kreisfreie Stadt', 'Aurich Landkreis', 'Bad Doberan Landkreis', 'Bad Dürkheim Landkreis', 'Bad Kissingen Landkreis', 'Bad Kreuznach Landkreis', 'Bad Tölz-Wolfratshausen Landkreis', 'Baden-Baden kreisfreie Stadt', 'Bamberg Landkreis', 'Bamberg kreisfreie Stadt', 'Barnim Landkreis', 'Bautzen Landkreis', 'Bayreuth Landkreis', 'Bayreuth kreisfreie Stadt', 'Berchtesgadener Land Landkreis', 'Bergstraße Landkreis', 'B

In [4]:
        
        self.specific_matches={'Altenkirchen (Westerwald)': ['LK Altenkirchen'],
                          'Amberg': ['SK Amberg'],
                          'Ansbach': ['LK Ansbach', 'SK Ansbach'],
                          'Aschaffenburg': ['LK Aschaffenburg', 'SK Aschaffenburg'],
                          'Augsburg': ['LK Augsburg', 'SK Augsburg'],
                          'Bamberg': ['LK Bamberg', 'SK Bamberg'],
                          'Bayreuth': ['SK Bayreuth', 'LK Bayreuth'],
                          'Brandenburg an der Havel': ['SK Brandenburg a.d.Havel'],
                          'Coburg': ['LK Coburg', 'SK Coburg'],
                          'Darmstadt': ['SK Darmstadt'],
                          'Dillingen a.d. Donau': ['LK Dillingen a.d.Donau'],
                          'Eifelkreis-Bitburg-Prüm': ['LK Bitburg-Prüm'],
                          'Eisenach': [],                # not available in the influenza location names
                          'Erlangen': ['SK Erlangen'],
                          'Flensburg': ['SK Flensburg'],
                          'Frankenthal (Pfalz)': ['SK Frankenthal'],
                          'Freiburg im Breisgau': ['SK Freiburg i.Breisgau'],
                          'Fürth': ['LK Fürth', 'SK Fürth'],
                          'Gera': ['SK Gera'],
                          'Halle (Saale)': ['SK Halle'],
                          'Heilbronn': ['LK Heilbronn', 'SK Heilbronn'],
                          'Hof': ['LK Hof', 'SK Hof'],
                          'Kaiserslautern': ['LK Kaiserslautern', 'SK Kaiserslautern'],
                          'Karlsruhe': ['LK Karlsruhe', 'SK Karlsruhe'],
                          'Kassel': ['LK Kassel', 'SK Kassel'],
                          'Kempten (Allgäu)': ['SK Kempten'],
                          'Koblenz': ['SK Koblenz'],
                          'Landau in der Pfalz': ['SK Landau i.d.Pfalz'],
                          'Landsberg am Lech': ['LK Landsberg a.Lech'],
                          'Landshut': ['LK Landshut', 'SK Landshut'],
                          'Leipzig': ['LK Leipzig', 'SK Leipzig'],
                          'Lindau (Bodensee)': ['LK Lindau'],
                          'Ludwigshafen am Rhein': ['SK Ludwigshafen'],
                          'Mainz': ['SK Mainz'],
                          'Mühldorf a. Inn': ['LK Mühldorf a.Inn'],
                          'Mülheim an der Ruhr': ['SK Mülheim a.d.Ruhr'],
                          'München': ['LK München', 'SK München'],
                          'Neumarkt i.d. OPf.': ['LK Neumarkt i.d.OPf.'],
                          'Neustadt a.d. Aisch-Bad Windsheim': ['LK Neustadt a.d.Aisch-Bad Windsheim'],
                          'Neustadt a.d. Waldnaab': ['LK Neustadt a.d.Waldnaab'],
                          'Neustadt an der Weinstraße': ['SK Neustadt a.d.Weinstraße'],
                          'Nürnberg': ['SK Nürnberg'], 
                          'Offenbach': ['LK Offenbach'], 
                          'Offenbach am Main': ['SK Offenbach'], 
                          'Oldenburg': ['LK Oldenburg'],
                          'Oldenburg (Oldenburg)': ['SK Oldenburg'],    
                          'Osnabrück': ['LK Osnabrück', 'SK Osnabrück'],
                          'Osterode am Harz': [],             # not available in the influenza location names
                          'Passau': ['LK Passau', 'SK Passau'],
                          'Pfaffenhofen a.d. Ilm': ['LK Pfaffenhofen a.d.Ilm'],
                          'Potsdam': ['SK Potsdam'],
                          'Regen': ['LK Regen'],
                          'Regensburg': ['LK Regensburg', 'SK Regensburg'],
                          'Rosenheim': ['LK Rosenheim', 'SK Rosenheim'],
                          'Rostock': ['LK Rostock', 'SK Rostock'],
                          'Schweinfurt': ['LK Schweinfurt', 'SK Schweinfurt'],
                          'St. Wendel': ['LK Sankt Wendel'],
                          'Straubing': ['SK Straubing'],
                          'Trier': ['SK Trier'],
                          'Ulm': ['SK Ulm'],
                          'Weiden i.d. OPf.': ['SK Weiden i.d.OPf.'], 
                          'Weimar': ['SK Weimar'],
                          'Worms': ['SK Worms'],
                          'Wunsiedel i. Fichtelgebirge': ['LK Wunsiedel i.Fichtelgebirge'],
                          'Würzburg': ['LK Würzburg', 'SK Würzburg']}

NameError: name 'self' is not defined